In [1]:
import sympy as sp

In [2]:
import matplotlib.pyplot as plt

In [3]:
import numpy as np

In [4]:
from sympy.abc import x

In [5]:
import unicodedata as ud

In [6]:
from sympy.solvers.solveset import linsolve

In [7]:
import sympy.core.relational as equal

In [8]:
import sympy.physics.units.quantities as sq

In [9]:
well_length = sq.Quantity( 'L' )

In [10]:
highest_potential = sq.Quantity( 'V_max' )

In [11]:
x = sp.Symbol( 'x' )

In [12]:
potential_energy = sp.Piecewise( ( highest_potential, x <= 0 ), ( 0, x < well_length ), ( highest_potential, x >= well_length ) )

In [13]:
test = sp.Symbol( 'psi' )

In [14]:
m = sq.Quantity( "m" )

In [15]:
psi_symbol = ud.lookup( "GREEK SMALL LETTER PSI" )

In [16]:
psi = sp.Function( "psi" ) # lambda g : test #sp.Function( "psi" )

In [17]:
hbar = sq.Quantity( "hbar" )

In [18]:
total_energy = sq.Quantity( "E" )

inverse_total_energy = 1.0 / total_energy

In [19]:
second_derivative = sp.Derivative( psi( x ), x, 2 )

In [20]:
make_shrodinger_left = lambda potential : inverse_total_energy * ( second_derivative + ( psi( x ) * potential ) )

In [21]:
shrodinger_left = make_shrodinger_left( potential_energy )

In [22]:
psi_solved = sp.solvers.ode.dsolve( shrodinger_left, psi( x ) )

In [23]:
psi_solved

Eq(psi(x), x**2*Piecewise((-V_max*C1/2, x <= 0), (0, x < L), (-V_max*C1/2, True)) + x**3*Piecewise((-V_max*C2/6, x <= 0), (0, x < L), (-V_max*C2/6, True)) + x**4*Piecewise((-V_max*Piecewise((-V_max*C1/2, x <= 0), (0, x < L), (-V_max*C1/2, True))/12, x <= 0), (0, x < L), (-V_max*Piecewise((-V_max*C1/2, x <= 0), (0, x < L), (-V_max*C1/2, True))/12, True)) + C2*x + C1 + O(x**6))

In [24]:
psi_solved.subs( x, 0 )

Eq(psi(0), C1)

In [25]:
sp.simplify( shrodinger_left )

Piecewise((1.0*(V_max*psi(x) + Derivative(psi(x), (x, 2)))/E, x <= 0), (1.0*Derivative(psi(x), (x, 2))/E, x < L), (1.0*(V_max*psi(x) + Derivative(psi(x), (x, 2)))/E, True))

In [26]:
shrodinger_left_outside_bounds = make_shrodinger_left( highest_potential )

In [27]:
shrodinger_left_inside_bounds = make_shrodinger_left( 0 )

In [28]:
left_physical_constraints = [
    shrodinger_left_outside_bounds.subs( x, well_length ), 
    shrodinger_left_outside_bounds.subs( x, 0 ), 
    shrodinger_left_outside_bounds.subs( x, well_length ), 
    ( shrodinger_left_outside_bounds * total_energy ), 
    ( shrodinger_left_inside_bounds * total_energy ), 
    sp.integrate( psi( x ) * sp.conjugate( psi( x ) ), ( x, 0, well_length ) ),  
]

In [29]:
right_physical_constraints = [ 
    shrodinger_left_inside_bounds.subs( x, well_length ), 
    shrodinger_left_inside_bounds.subs( x, 0 ), 
    shrodinger_left_outside_bounds.subs( x, 0 ), 
    ( total_energy * psi( x ) ), 
    ( total_energy * psi( x ) ), 
    1 
]

In [30]:
equations = []
#solutions = [ psi( well_length ), psi( 0 ), psi( 0 ), psi( x ), psi( x ), psi( x ) ]
solutions = []

for i in range( len( right_physical_constraints ) ): 
    #equations.append( sp.Eq( left_physical_constraints[ i ], right_physical_constraints[ i ] ) )
    #solutions.append( psi( x ) )
    #equations.append( left_physical_constraints[ i ] - right_physical_constraints[ i ] )
    #solutions.append( 1 if i == len( right_physical_constraints ) - 1 else 0 )
    equations.append( left_physical_constraints[ i ] ) #.subs( m, 5 ).subs( well_length, 3 ).subs( total_energy, 7 ).subs( highest_potential, 17 ) )
    solutions.append( right_physical_constraints[ i ] )#1 if i == len( right_physical_constraints ) - 1 else right_physical_constraints[ i ].subs( m, 5 ).subs( well_length, 3 ).subs( total_energy, 7 ).subs( highest_potential, 17 ) )
    #print( equations[ -1 ].is_constant() )
    #print( solutions[ -1 ].is_constant() )

#solutions[ -1 ] = 1

In [31]:
solutions[ 5 ]

1

In [32]:
equations

[1.0*(V_max*psi(L) + Derivative(psi(L), (L, 2)))/E,
 1.0*(V_max*psi(0) + Subs(Derivative(psi(x), (x, 2)), x, 0))/E,
 1.0*(V_max*psi(L) + Derivative(psi(L), (L, 2)))/E,
 1.0*V_max*psi(x) + 1.0*Derivative(psi(x), (x, 2)),
 1.0*Derivative(psi(x), (x, 2)),
 Integral(psi(x)*conjugate(psi(x)), (x, 0, L))]

In [33]:
linsolve( tuple( equations ), tuple( solutions ) )

EmptySet

In [34]:
sp.solve( equations, solutions )

[]

In [35]:
tuple( solutions )

(1.0*Derivative(psi(L), (L, 2))/E,
 1.0*Subs(Derivative(psi(x), (x, 2)), x, 0)/E,
 1.0*(V_max*psi(0) + Subs(Derivative(psi(x), (x, 2)), x, 0))/E,
 E*psi(x),
 E*psi(x),
 1)

In [36]:
equations

[1.0*(V_max*psi(L) + Derivative(psi(L), (L, 2)))/E,
 1.0*(V_max*psi(0) + Subs(Derivative(psi(x), (x, 2)), x, 0))/E,
 1.0*(V_max*psi(L) + Derivative(psi(L), (L, 2)))/E,
 1.0*V_max*psi(x) + 1.0*Derivative(psi(x), (x, 2)),
 1.0*Derivative(psi(x), (x, 2)),
 Integral(psi(x)*conjugate(psi(x)), (x, 0, L))]

In [37]:
psi_ode = sp.solvers.ode.dsolve( equations, solutions )

ValueError: 
dsolve_system can solve a system of ODEs with only one independent
variable.